# Analysis in R

In [ ]:
library(rgbif)
library(CoordinateCleaner)
library(dplyr)
library(ggplot2)
library(rgbif)
library(sp)
library(data.table)
library(plotbiomes)
library(raster)
library(maptools)


## Load data frame GBIF

In [ ]:
meta_gbif=read.csv('./GBIF/For_gbif_trickstar.xlsx - Sheet1.csv') 
GBIF_extract2=function(target){
    extract=meta_gbif[meta_gbif$Category==target,]
    N=nrow(extract)
    print(paste0("We have ", N, " species"))
    for(i in 4:N){
        species=extract[i,3]
        print(species)
        taxonkey=name_backbone(species)$usageKey
        Load=occ_download(pred("taxonKey", taxonkey),format='SIMPLE_CSV', 
                      user='fill your user name', pwd='fill your password', email='fill you email')
        print(Load)
    }
}
#example
#GBIF_extract2('skunk')

## GBIF cleaning

In [ ]:
GBIF_cleaning= function(target){
    #sop_name: str of species name
    meta_gbif=read.csv('./GBIF/For_gbif_trickstar.xlsx - Sheet1.csv') # which species corresponds which trickster
    extract=meta_gbif[meta_gbif$Category==target,]
    N=nrow(extract)
    for(i in 1:N){
        species=extract[i,3]
        print(species)
        path=paste0('./GBIF/',target, '/', species, '.csv')
        #data= read.csv(path, sep='	')
        data=fread(file=path, sep='	')
        # remove records without coordinates
        data <- data%>%
          filter(!is.na(decimalLongitude))%>%
          filter(!is.na(decimalLatitude))

        data <- data.frame(data)
        # there might be non-numeric elements in lon and lat
        data$"decimalLongitude"=as.numeric(as.character(data$"decimalLongitude"))
        data$"decimalLatitude"=as.numeric(as.character(data$"decimalLatitude"))
        flags <- clean_coordinates(x = data,
                                   lon = "decimalLongitude",
                                   lat = "decimalLatitude",
                                   countries = "countryCode",
                                   species = "species",
                                  tests = c("capitals", "centroids", "gbif", "institutions",
                                            "zeros")) # most test are on by default

        #Exclude problematic records
        data_cl <- data[flags$.summary,]
        # we need only GIS data
        path_save=paste0('./GBIF/',target, '/', species, '_cleaned.csv')
        write.csv(data_cl[, c("decimalLongitude","decimalLatitude")]
          ,path_save, row.names = FALSE) 
        }
    }
#example
GBIF_cleaning('racoon')

## Counting the number of data after cleaning

In [11]:
TS=read.csv('TrickSter_data3.csv')
Species=unique(TS$TrickSter)
count=c()
for (i in 1:length(Species))
{
    count=c(count, sum(TS$TrickSter==Species[i]))

}
TS_count=data.frame(Species, count)
TS_count

Species,count
<chr>,<int>
raven,79
skunk,12
mink,9
rabbit,190
owl,14
hawk,13
water bird,12
opossum,25
rat,17


In [30]:
real_animal_mweta=read.csv('./GBIF/For_gbif_trickstar.xlsx - Sheet1.csv')
count=c()
for (i in 1:length(Species))
{
    print(Species[i]) 
    if (Species[i] != 'water bird' && Species[i] != 'monkey')
        {
           
        Taxa=real_animal_meta[real_animal_meta$Category == Species[i], ]$Taxa
        print(Taxa)
            num=0
            for (j in 1:length(Taxa))
                {
                    data=read.csv(paste0('./GBIF/', Species[i], '/', Taxa[j], '_cleaned.csv'))
                    num=num+nrow(data)
                }
            count=c(count, num)
        }
    else{
        count=c(count, 0) # use 0 instead of NaN
        }
}
RA_count=data.frame(Species, count)
RA_count

[1] "raven"
[1] "Corvus albicollis"    "Corvus corax"         "Corvus coronoides"   
[4] "Corvus crassirostris" "Corvus cryptoleucus"  "Corvus mellori "     
[7] "Corvus rhipidurus"    "Corvus ruficollis"   
[1] "skunk"
[1] "Conepatus" "Mephitis"  "Spilogale"
[1] "mink"
[1] "Neogale vison"    "Mustela lutreola"
[1] "rabbit"
[1] "Leporidae"
[1] "owl"
[1] "Strigiformes"
[1] "hawk"
[1] "Accipitridae"
[1] "water bird"
[1] "opossum"
[1] "Didelphimorphia"
[1] "rat"
[1] "Rattus"    "Neotoma"   "Bandicota" "Dipodomys"
[1] "mouse"
[1] "Mus"        "Peromyscus"
[1] "ground squirrel"
[1] "Xerus"           "Geosciurus"      "Euxerus"         "Spermophilopsis"
[5] "Atlantoxerus"   
[1] "monkey"
[1] "spider"
[1] "Araneae"
[1] "raccoon"
[1] "Procyon lotor"
[1] "anteater"
[1] "Myrmecophaga tridactyla" "Cyclopes didactylus"    
[3] "Tamandua tetradactyla"   "Tamandua mexicana"      
[1] "porcupine"
[1] "Hystricidae"    "Erethizontidae"
[1] "badger"
 [1] "Arctonyx albogularis"   "Arctonyx collaris"     

Species,count
<chr>,<dbl>
raven,8642919
skunk,27698
mink,10353
rabbit,1313595
owl,4273726
hawk,50898205
water bird,0
opossum,59114
rat,325063


In [32]:
print(min(RA_count$count))
print(max(RA_count$count))

[1] 0
[1] 50898205


## Biome classification and plots

In [26]:
data=read.csv("Biome_distributions.csv") # distributions of RA and TS
data_null=read.csv("Real_animal_hex_biome.csv") # distributions of RA _null
data$Annu_Prec=data$Annu_Prec/10 # convert mm to cm
data_null$Annu_Prec=data_null$Annu_Prec/10 # convert mm to cm

[1] NaN

In [ ]:
Biome_class=function(data){
    df1=Whittaker_biomes[Whittaker_biomes$biome_id==1, ]
    pol1.x=df1$temp_c
    pol1.y=df1$precp_cm
    
    df2=Whittaker_biomes[Whittaker_biomes$biome_id==2, ]
    pol2.x=df2$temp_c
    pol2.y=df2$precp_cm
    
    df3=Whittaker_biomes[Whittaker_biomes$biome_id==3, ]
    pol3.x=df3$temp_c
    pol3.y=df3$precp_cm
    
    df4=Whittaker_biomes[Whittaker_biomes$biome_id==4, ]
    pol4.x=df4$temp_c
    pol4.y=df4$precp_cm
    
    df5=Whittaker_biomes[Whittaker_biomes$biome_id==5, ]
    pol5.x=df5$temp_c
    pol5.y=df5$precp_cm
    
    df6=Whittaker_biomes[Whittaker_biomes$biome_id==6, ]
    pol6.x=df6$temp_c
    pol6.y=df6$precp_cm
    
    df7=Whittaker_biomes[Whittaker_biomes$biome_id==7, ]
    pol7.x=df7$temp_c
    pol7.y=df7$precp_cm
    
    df8=Whittaker_biomes[Whittaker_biomes$biome_id==8, ]
    pol8.x=df8$temp_c
    pol8.y=df8$precp_cm
    
    df9=Whittaker_biomes[Whittaker_biomes$biome_id==9, ]
    pol9.x=df9$temp_c
    pol9.y=df9$precp_cm
    
    category=c()
    X=data$Annu_Mean_Temp
    Y=data$Annu_Prec # we need cm, not mm
    for (i in 1:nrow(data)){
        X=data$Annu_Mean_Temp[i]
        Y=data$Annu_Prec[i]
        if (point.in.polygon(X, Y, pol1.x, pol1.y)==1){
            category=c(category, 1)
        }
        else 
            if (point.in.polygon(X, Y, pol2.x, pol2.y)==1){
            category=c(category, 2)
        }
        else if (point.in.polygon(X, Y, pol3.x, pol3.y)==1){
            category=c(category, 3)
        }
        else if (point.in.polygon(X, Y, pol4.x, pol4.y)==1){
            category=c(category, 4)
        }
        else if (point.in.polygon(X, Y, pol5.x, pol5.y)==1){
            category=c(category, 5)
        }
        else if (point.in.polygon(X, Y, pol6.x, pol6.y)==1){
            category=c(category, 6)
        }
        else if (point.in.polygon(X, Y, pol7.x, pol7.y)==1){
            category=c(category, 7)
        }
        else if (point.in.polygon(X, Y, pol8.x, pol8.y)==1){
            category=c(category, 8)
        }
        else if (point.in.polygon(X, Y, pol9.x, pol9.y)==1){
            category=c(category, 9)
        }
        else{
        
            category=c(category, 10)
        }
        
    }
    #print(category)
    data$biome_id=as.factor(category)
    return(data)
}
data=Biome_class(data)
head(data)

In [ ]:
g=whittaker_base_plot()+theme(legend.position="none")+ylim(-10, 450)
g=g+geom_point(data=data_null, aes(Annu_Mean_Temp, Annu_Prec), 
               size=4, colour='black', alpha=0.0, shape=4) # null data
g=g+labs(x='Annual mean temperature (°C)', y='Precipitation (cm/year)')
 g=g+theme(
     legend.justification = c(0, 1), # pick the upper left corner of the legend box and
    legend.position = c(0, 1), # adjust the position of the corner as relative to axis
    legend.background = element_rect(fill = NA), # transparent legend background
    legend.box = "horizontal", # horizontal arrangement of multiple legends
    legend.spacing.x = unit(0.5, units = "cm"), # horizontal spacing between legends
    legend.text=element_text(size=36), 
    legend.title=element_text(size=36), 
     axis.text = element_text(size = 36),
        axis.title = element_text(size = 48),
        panel.grid = element_blank() # eliminate grids
      )
    #
#g=g+guides(color = guide_legend(nrow = 6)) 
ggsave(paste0('./Biome/Biome_example.pdf'), width = 18, height = 12)
g

In [ ]:
BiomePlot=function(target){
    g=whittaker_base_plot()+theme(legend.position="none")
    
    g=g+geom_point(data=data_null, aes(Annu_Mean_Temp, Annu_Prec), 
               size=4, colour='black', alpha=0.0, shape=4) # null data
    
    
    d=data[data$index=='RA', ]
    g=g+geom_point(data=d[d$Species==target, ], 
                aes(Annu_Mean_Temp, Annu_Prec), 
                shape  = 1,
               stroke = 2, # acts as the thickness of the boundary line
                 colour = 'black', # acts as the color of the boundary line
             size   = 9)
     g=g+geom_point(data=d[d$Species==target, ], 
                aes(Annu_Mean_Temp, Annu_Prec), 
                shape  = 1,
               stroke = 1, # acts as the thickness of the boundary line
                 colour = '#8da0cb', # acts as the color of the boundary line
             size   = 8)
    g=g+geom_point(data=data[data$Species==target, ], 
                   aes(Annu_Mean_Temp, Annu_Prec, color=index, shape=index, size=index), 
                  )+ scale_color_manual(values=c('#8da0cb', '#fc8d62'))+scale_size_manual(values=c(8,8))
    g=g+ylim(-10, 450)
    #+ theme(axis.title = element_text(size = 20), axis.text= element_text(size = 16))     
    #labs(x='Annual mean temperature (°C)', y='Precipitation (cm/year)')++guides(color = guide_legend(nrow = 6), legend.position=c(0.4, 1.12)) 
    
    d=data[data$index=='TS', ]
    g=g+geom_point(data=d[d$Species==target, ], 
                aes(Annu_Mean_Temp, Annu_Prec), 
                shape  = 2,
               stroke = 3, # acts as the thickness of the boundary line
                 colour = 'gray95', # acts as the color of the boundary line
             size   = 9)
     g=g+geom_point(data=d[d$Species==target, ], 
                aes(Annu_Mean_Temp, Annu_Prec), 
                shape  = 17,
               stroke = 1, # acts as the thickness of the boundary line
                 colour = '#fc8d62', # acts as the color of the boundary line
             size   = 8)
    #g=g+theme_bw()+ggtitle(target) +
      g=g+theme(
        #legend.justification = c(0, 1), # pick the upper left corner of the legend box and
        #legend.position = c(0, 1), # adjust the position of the corner as relative to axis
        #legend.background = element_rect(fill = NA), # transparent legend background
        #legend.box = "horizontal", # horizontal arrangement of multiple legends
        #legend.spacing.x = unit(0.5, units = "cm"), # horizontal spacing between legends
        #legend.text=element_text(size=36),
        #legend.title=element_text(size=36),
        axis.text = element_text(size = 36),
        axis.title = element_blank(),
          #plot.title=element_text(size=64),
        panel.grid = element_blank() # eliminate grids
      )
    ggsave(paste0('./Biome/Biome_', target, '.pdf'), width = 18, height = 12)
    return (g)
}

In [ ]:
s=unique(data$Species)
for (i in 1:length(s)){
    target=s[i]
    g=BiomePlot(target)
    print(g)
    }

## Comparison of biome classes between real animal and null model

In [ ]:
data_null=read.csv("Real_animal_hex_biome.csv")
data=read.csv("Biome_distributions.csv") # THIS  FILE MAY LACK SOME DATA. NEED TO CHECK TOMORROW
data_RA=data[data$index=='RA',]
Biome_class=function(data){
    df1=Whittaker_biomes[Whittaker_biomes$biome_id==1, ]
    pol1.x=df1$temp_c
    pol1.y=df1$precp_cm
    
    df2=Whittaker_biomes[Whittaker_biomes$biome_id==2, ]
    pol2.x=df2$temp_c
    pol2.y=df2$precp_cm
    
    df3=Whittaker_biomes[Whittaker_biomes$biome_id==3, ]
    pol3.x=df3$temp_c
    pol3.y=df3$precp_cm
    
    df4=Whittaker_biomes[Whittaker_biomes$biome_id==4, ]
    pol4.x=df4$temp_c
    pol4.y=df4$precp_cm
    
    df5=Whittaker_biomes[Whittaker_biomes$biome_id==5, ]
    pol5.x=df5$temp_c
    pol5.y=df5$precp_cm
    
    df6=Whittaker_biomes[Whittaker_biomes$biome_id==6, ]
    pol6.x=df6$temp_c
    pol6.y=df6$precp_cm
    
    df7=Whittaker_biomes[Whittaker_biomes$biome_id==7, ]
    pol7.x=df7$temp_c
    pol7.y=df7$precp_cm
    
    df8=Whittaker_biomes[Whittaker_biomes$biome_id==8, ]
    pol8.x=df8$temp_c
    pol8.y=df8$precp_cm
    
    df9=Whittaker_biomes[Whittaker_biomes$biome_id==9, ]
    pol9.x=df9$temp_c
    pol9.y=df9$precp_cm
    
    category=c()
    X=data$Annu_Mean_Temp
    Y=data$Annu_Prec 
    for (i in 1:nrow(data)){
        X=data$Annu_Mean_Temp[i]
        Y=data$Annu_Prec[i]/10 # convert mm to cm
        if (point.in.polygon(X, Y, pol1.x, pol1.y)==1){
            category=c(category, 1)
        }
        else 
            if (point.in.polygon(X, Y, pol2.x, pol2.y)==1){
            category=c(category, 2)
        }
        else if (point.in.polygon(X, Y, pol3.x, pol3.y)==1){
            category=c(category, 3)
        }
        else if (point.in.polygon(X, Y, pol4.x, pol4.y)==1){
            category=c(category, 4)
        }
        else if (point.in.polygon(X, Y, pol5.x, pol5.y)==1){
            category=c(category, 5)
        }
        else if (point.in.polygon(X, Y, pol6.x, pol6.y)==1){
            category=c(category, 6)
        }
        else if (point.in.polygon(X, Y, pol7.x, pol7.y)==1){
            category=c(category, 7)
        }
        else if (point.in.polygon(X, Y, pol8.x, pol8.y)==1){
            category=c(category, 8)
        }
        else if (point.in.polygon(X, Y, pol9.x, pol9.y)==1){
            category=c(category, 9)
        }
        else{
        
            category=c(category, 10)
        }
        
    }
    #print(category)
    data$biome_id=as.factor(category)
    return(data)
}
data=Biome_class(data_null)

In [ ]:
vals=c()
species=unique(data_RA$Species)
for (i in 1:length(species))
    {
    print(species[i])
    
    d=data_RA[data_RA$Species==species[i],]
    d=Biome_class(d)
    print(nrow(d))
    freq=c(nrow(d[d$biome_id==1,]),nrow(d[d$biome_id==2,]),
           nrow(d[d$biome_id==3,]), nrow(d[d$biome_id==4,]),
           nrow(d[d$biome_id==5,]), nrow(d[d$biome_id==6,]),
           nrow(d[d$biome_id==7,]),nrow(d[d$biome_id==8,]),
           nrow(d[d$biome_id==9,]),nrow(d[d$biome_id==10,]))
    print(freq/sum(freq))
    #ans= multinomial.test(freq, p_null, MonteCarlo = TRUE)      
    ans=chisq.test(freq, p=p_null, B=1000000) # because of rare data, chi squre data seems problem use multi-nominal test
    #print(ans)
    pvals=c(pvals, ans$p.value)
    }
p.adjust(pvals, 'fdr')

## Comparison of biome classes between trickster animals and null model

In [ ]:
data=read.csv("Biome_distributions.csv") # THIS  FILE MAY LACK SOME DATA. NEED TO CHECK TOMORROW
data_TS=data[data$index=='TS',]
pvals=c()
species=unique(data_TS$Species)
for (i in 1:length(species))
    {
    print(species[i])
    
    d=data_TS[data_TS$Species==species[i],]
    d=Biome_class(d)
    print(nrow(d))
    freq=c(nrow(d[d$biome_id==1,]),nrow(d[d$biome_id==2,]),
           nrow(d[d$biome_id==3,]), nrow(d[d$biome_id==4,]),
           nrow(d[d$biome_id==5,]), nrow(d[d$biome_id==6,]),
           nrow(d[d$biome_id==7,]),nrow(d[d$biome_id==8,]),
           nrow(d[d$biome_id==9,]),nrow(d[d$biome_id==10,]))
    print(freq/sum(freq))
    #ans= multinomial.test(freq, p_null, MonteCarlo = TRUE)      
    ans=chisq.test(freq, p=p_null, B=1000000) # because of rare data, chi squre data seems problem use multi-nominal test
    print(ans)
    pvals=c(pvals, ans$p.value)
    }
p.adjust(pvals, 'fdr')

## Comparison of biome classes between real and trickster animals

In [ ]:
pvals=c()
for (i in 1:length(s)){
    target=s[i]
    print(target)
    d=data[data$Species== target, ]
    
    d_RA=d[d$index=='RA', ]
    freq_RA=c(nrow(d_RA[d_RA$biome_id==1,]),nrow(d_RA[d_RA$biome_id==2,]),
           nrow(d_RA[d_RA$biome_id==3,]), nrow(d_RA[d_RA$biome_id==4,]),
           nrow(d_RA[d_RA$biome_id==5,]), nrow(d_RA[d_RA$biome_id==6,]),
           nrow(d_RA[d_RA$biome_id==7,]),nrow(d_RA[d_RA$biome_id==8,]),
           nrow(d_RA[d_RA$biome_id==9,]),nrow(d_RA[d_RA$biome_id==10,]))
    freq_RA=freq_RA/sum(freq_RA)
    d_TS=d[d$index=='TS', ]
    freq_TS=c(nrow(d_TS[d_TS$biome_id==1,]),nrow(d_TS[d_TS$biome_id==2,]),
           nrow(d_TS[d_TS$biome_id==3,]), nrow(d_TS[d_TS$biome_id==4,]),
           nrow(d_TS[d_TS$biome_id==5,]), nrow(d_TS[d_TS$biome_id==6,]),
           nrow(d_TS[d_TS$biome_id==7,]),nrow(d_TS[d_TS$biome_id==8,]),
           nrow(d_TS[d_TS$biome_id==9,]),nrow(d_TS[d_TS$biome_id==10,]))
    drop=c()
    for (i in 1:10){
       if (freq_TS[i]==0 && freq_RA[i]==0){
           drop=c(drop, i)
       }
    }


    if(length(drop)>0){
        ans=chisq.test(freq_TS[-c(drop)], p=freq_RA[-c(drop)], B=1000000)
        } else{
    ans=chisq.test(freq_TS, p=freq_RA, B=1000000)
    }  
    print(ans)
    pvals=c(pvals, ans$p.value)
}
#print(pvals)
p.adjust(pvals, 'fdr')